In [4]:
import requests

def human_readable_volume(volume):
    """
    Converts a volume number into a more human-readable format with suffixes like K, M, B, etc.

    Parameters:
    - volume (float): The volume to be formatted.

    Returns:
    - str: The volume in a human-readable format.
    """
    for unit in ['', 'K', 'M', 'B', 'T']:
        if abs(volume) < 1000:
            return f"{volume:.2f}{unit}"
        volume /= 1000
    return f"{volume:.2f}P"  # Fallback to petabytes if the number is too large

def find_tokens_with_good_volume(threshold=0.5, volume_threshold=100000):
    """
    Finds all tokens on Binance traded against USDT with a price below the specified threshold and with trading volume considered as 'good', 
    and formats the volume into a human-readable form.

    Parameters:
    - threshold (float): The price threshold to filter tokens by.
    - volume_threshold (float): The minimum trading volume to filter tokens by.

    Returns:
    - List of symbols (str) with prices below the specified threshold and trading volume above the specified threshold, with human-readable volume.
    """

    # Endpoint URL for fetching 24-hour trading data for all symbols
    api_url = "https://api.binance.com/api/v3/ticker/24hr"
    
    # Fetch the 24-hour trading data for all symbols
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        
        # Filter symbols based on the criteria
        qualified_tokens = [item for item in data if 
                            item['symbol'].endswith('USDT') and 
                            float(item['lastPrice']) < threshold and 
                            float(item['quoteVolume']) > volume_threshold]
        
        # Sort the filtered tokens by price from higher to lower
        sorted_qualified_tokens = sorted(qualified_tokens, key=lambda x: float(x['quoteVolume']), reverse=True)
        
        # Format the volume for each token into a human-readable form
        for token in sorted_qualified_tokens:
            token['quoteVolume'] = human_readable_volume(float(token['quoteVolume']))
        
        return sorted_qualified_tokens
    else:
        print("Failed to fetch data from Binance API")
        return []

# Example usage - Adjust the volume threshold as needed
tokens = find_tokens_with_good_volume(0.5, 100000)  # Here 100000 is just an example threshold for "good" volume
for token in tokens:
    print(f"{token['symbol']}: Last Price {token['lastPrice']}, Volume {token['quoteVolume']}")


BOMEUSDT: Last Price 0.01190300, Volume 1.20B
PEPEUSDT: Last Price 0.00000626, Volume 472.72M
DOGEUSDT: Last Price 0.13242000, Volume 449.42M
SHIBUSDT: Last Price 0.00002544, Volume 395.43M
FLOKIUSDT: Last Price 0.00017149, Volume 209.90M
BONKUSDT: Last Price 0.00002116, Volume 97.65M
1000SATSUSDT: Last Price 0.00044220, Volume 92.22M
GALAUSDT: Last Price 0.05336000, Volume 90.48M
TRXUSDT: Last Price 0.12106000, Volume 75.26M
GRTUSDT: Last Price 0.37960000, Volume 74.11M
CFXUSDT: Last Price 0.36980000, Volume 71.07M
VANRYUSDT: Last Price 0.26452000, Volume 56.74M
LUNCUSDT: Last Price 0.00012906, Volume 54.76M
ALTUSDT: Last Price 0.44324000, Volume 48.43M
MEMEUSDT: Last Price 0.03364900, Volume 48.32M
XLMUSDT: Last Price 0.12440000, Volume 41.64M
KEYUSDT: Last Price 0.00756500, Volume 39.66M
VETUSDT: Last Price 0.03926000, Volume 36.36M
GMTUSDT: Last Price 0.26800000, Volume 36.16M
HBARUSDT: Last Price 0.10400000, Volume 33.68M
ALGOUSDT: Last Price 0.23130000, Volume 33.59M
CHZUSDT: Las

In [11]:
import requests

def get_binance_symbols():
    """
    Fetches all trading symbols from Binance.
    """
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)
    symbols = [item['symbol'] for item in response.json()['symbols'] if 'USDT' in item['symbol']]  # Filter for USDT pairs
    return symbols

def search_coingecko_id(symbol):
    """
    Searches for a CoinGecko coin ID using a symbol.
    """
    url = "https://api.coingecko.com/api/v3/search"
    params = {'query': symbol.replace('USDT', '')}  # Remove USDT for the search
    response = requests.get(url, params=params)
    results = response.json()['coins']
    if results:
        # Assume the first result is the correct one
        return results[0]['id']
    return None

def get_circulating_supply(coin_gecko_id):
    """
    Fetches the circulating supply for a given CoinGecko coin ID.
    """
    if not coin_gecko_id:
        return None
    url = f"https://api.coingecko.com/api/v3/coins/{coin_gecko_id}"
    response = requests.get(url)
    data = response.json()
    return data['market_data']['circulating_supply']

# Example usage
binance_symbols = get_binance_symbols()[:10]  # Limit to first 10 for demonstration
for symbol in binance_symbols:
    coin_gecko_id = search_coingecko_id(symbol)
    if coin_gecko_id:
        supply = get_circulating_supply(coin_gecko_id)
        print(f"{symbol} (CoinGecko ID: {coin_gecko_id}): Circulating Supply = {supply}")
    else:
        print(f"Could not find CoinGecko ID for {symbol}")


BTCUSDT (CoinGecko ID: bitcoin): Circulating Supply = 19656937.0
ETHUSDT (CoinGecko ID: ethereum): Circulating Supply = 120079481.714212
BNBUSDT (CoinGecko ID: binancecoin): Circulating Supply = 153856150.0
BCCUSDT (CoinGecko ID: tbcc): Circulating Supply = 0.0
NEOUSDT (CoinGecko ID: neo): Circulating Supply = 70530000.0


JSONDecodeError: Expecting value: line 1 column 1 (char 0)